In [21]:
import pandas as pd #Importação da biblioteca "Pandas"

df = pd.read_csv("/content/drive/MyDrive/ocorrencia.csv", sep=";", parse_dates=["ocorrencia_dia"], dayfirst=True)

In [3]:
df #Primeira verificação geral de inconsistências

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,BRASIL,****,2010-01-03,12:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,BRASIL,SBBE,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,BRASIL,SBRJ,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,BRASIL,****,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,BRASIL,SBPK,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,80245,80245,80245,80245,80245,INCIDENTE,-8.12638,-34.92277,RECIFE,PE,BRASIL,SBRF,2021-08-15,15:05:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
6110,80247,80247,80247,80247,80247,INCIDENTE,-8.713611111111,-63.90277777777,PORTO VELHO,RO,BRASIL,SBPV,2021-08-16,02:30:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
6111,80251,80251,80251,80251,80251,INCIDENTE,-1.384722222222,-48.47888888888,BELÉM,PA,BRASIL,SBBE,2021-08-17,15:45:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
6112,80257,80257,80257,80257,80257,ACIDENTE,-15.430833,-54.699722,DOM AQUINO,MT,BRASIL,****,2021-08-18,16:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO


In [4]:
df.dtypes #Verificação dos tipos de dados

codigo_ocorrencia                          int64
codigo_ocorrencia1                         int64
codigo_ocorrencia2                         int64
codigo_ocorrencia3                         int64
codigo_ocorrencia4                         int64
ocorrencia_classificacao                  object
ocorrencia_latitude                       object
ocorrencia_longitude                      object
ocorrencia_cidade                         object
ocorrencia_uf                             object
ocorrencia_pais                           object
ocorrencia_aerodromo                      object
ocorrencia_dia                    datetime64[ns]
ocorrencia_hora                           object
investigacao_aeronave_liberada            object
investigacao_status                       object
divulgacao_relatorio_numero               object
divulgacao_relatorio_publicado            object
divulgacao_dia_publicacao                 object
total_recomendacoes                        int64
total_aeronaves_envo

In [ ]:
pd.unique(df['ocorrencia_aerodromo']) 
#Comando usado para verificar inconsistências por colunas
#Não é necessári usar o método em colunas únicas

In [ ]:
sorted(pd.unique(df['ocorrencia_aerodromo'])) 
#Exibe os comandos de forma classificada

Inconsistências gerais:

* "*****", "####", "###!",

In [22]:
backup = df.copy()

In [28]:
df.replace(["**", "***", "****", "*****", "####", "###!"], pd.NA, inplace=True) #Substitindo dados inconsistentes por "NA"
#####Inserir valores nulos em espaços reservados para números#####

In [9]:
df

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,<NA>,<NA>,RIO DE JANEIRO,RJ,BRASIL,<NA>,2010-01-03,12:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,BRASIL,SBBE,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,BRASIL,SBRJ,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,BRASIL,<NA>,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,BRASIL,SBPK,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,80245,80245,80245,80245,80245,INCIDENTE,-8.12638,-34.92277,RECIFE,PE,BRASIL,SBRF,2021-08-15,15:05:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
6110,80247,80247,80247,80247,80247,INCIDENTE,-8.713611111111,-63.90277777777,PORTO VELHO,RO,BRASIL,SBPV,2021-08-16,02:30:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
6111,80251,80251,80251,80251,80251,INCIDENTE,-1.384722222222,-48.47888888888,BELÉM,PA,BRASIL,SBBE,2021-08-17,15:45:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
6112,80257,80257,80257,80257,80257,ACIDENTE,-15.430833,-54.699722,DOM AQUINO,MT,BRASIL,<NA>,2021-08-18,16:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO


In [10]:
df.isna().sum() #Contagem dos campos nulos

codigo_ocorrencia                    0
codigo_ocorrencia1                   0
codigo_ocorrencia2                   0
codigo_ocorrencia3                   0
codigo_ocorrencia4                   0
ocorrencia_classificacao             0
ocorrencia_latitude               2290
ocorrencia_longitude              2292
ocorrencia_cidade                    0
ocorrencia_uf                        2
ocorrencia_pais                      0
ocorrencia_aerodromo              2456
ocorrencia_dia                       0
ocorrencia_hora                      1
investigacao_aeronave_liberada    2399
investigacao_status                341
divulgacao_relatorio_numero       3920
divulgacao_relatorio_publicado       0
divulgacao_dia_publicacao         4537
total_recomendacoes                  0
total_aeronaves_envolvidas           0
ocorrencia_saida_pista               0
dtype: int64

In [11]:
#df.drop_duplicates() #dropa as linhas duplicadas

In [12]:
#pip install pandera #Instalando pandera

In [13]:
import pandera as pa #Importando pandera como "pa"

Validação dos Dados

In [14]:
schema = pa.DataFrameSchema(
    columns = { 
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String,pa.Check.str_length(2,2),nullable=True),
        "ocorrencia_aerodromo":pa.Column(pa.String,pa.Check.str_length(4,4),nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String,pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$'),nullable=True),
        "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [16]:
schema.validate(df) #Validando esquema aplicao ao dataframe

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,<NA>,<NA>,RIO DE JANEIRO,RJ,BRASIL,<NA>,2010-01-03,12:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,BRASIL,SBBE,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,BRASIL,SBRJ,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,BRASIL,<NA>,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,BRASIL,SBPK,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,80245,80245,80245,80245,80245,INCIDENTE,-8.12638,-34.92277,RECIFE,PE,BRASIL,SBRF,2021-08-15,15:05:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
6110,80247,80247,80247,80247,80247,INCIDENTE,-8.713611111111,-63.90277777777,PORTO VELHO,RO,BRASIL,SBPV,2021-08-16,02:30:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
6111,80251,80251,80251,80251,80251,INCIDENTE,-1.384722222222,-48.47888888888,BELÉM,PA,BRASIL,SBBE,2021-08-17,15:45:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
6112,80257,80257,80257,80257,80257,ACIDENTE,-15.430833,-54.699722,DOM AQUINO,MT,BRASIL,<NA>,2021-08-18,16:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO


Filtragem de Dados

In [17]:
filter = df.total_recomendacoes > 11 #Traz todas as linhas que possuem recomendação maior do que 11

df.loc[filter]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
326,42250,42250,42250,42250,42250,ACIDENTE,-22.9102777778,-43.1627777778,RIO DE JANEIRO,RJ,BRASIL,SBRJ,2010-08-12,12:26:00,SIM,FINALIZADA,A-001/CENIPA/2012,SIM,2012-02-22,19,1,SIM
872,44796,44796,44796,44796,44796,ACIDENTE,-8.1522222222,34.9094444444,RECIFE,PE,BRASIL,SBRF,2011-07-13,09:54:00,SIM,FINALIZADA,A-019/CENIPA/2013,SIM,2013-07-18,23,1,NÃO
1062,45554,45554,45554,45554,45554,INCIDENTE GRAVE,-18.9669444444,-48.2252777778,UBERLÂNDIA,MG,BRASIL,SBUL,2011-11-15,02:15:00,NaN,FINALIZADA,IG-018/CENIPA/2013,SIM,2013-07-16,20,1,SIM
2797,52265,52265,52265,52265,52265,ACIDENTE,-23.9597222222,-46.3269444444,SANTOS,SP,BRASIL,<NA>,2014-08-13,13:03:00,NaN,FINALIZADA,A-134/CENIPA/2014,SIM,2016-01-11,13,1,NÃO
3932,66432,66432,66432,66432,66432,INCIDENTE GRAVE,-20.3861111111,-40.4888888889,VITÓRIA,ES,BRASIL,<NA>,2017-02-21,11:47:00,NaN,FINALIZADA,IG-031/CENIPA/2017,SIM,2020-10-02,12,1,NÃO


In [18]:
df.loc[filter, ["ocorrencia_cidade", "ocorrencia_dia"]] #Pega apenas a cidade e o dia da ocorrencia que possuem recomendação maior do que 11

,ocorrencia_cidade,ocorrencia_dia
326,RIO DE JANEIRO,2010-08-12
872,RECIFE,2011-07-13
1062,UBERLÂNDIA,2011-11-15
2797,SANTOS,2014-08-13
3932,VITÓRIA,2017-02-21


In [19]:
filter1 = df.ocorrencia_classificacao == "ACIDENTE"
filter2 = df.ocorrencia_uf == "RJ"
df.loc[filter1 & filter2] #Utilizando dois filtros #& equivale ao "AND" #| equivale a "OR"

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
31,39323,39323,39323,39323,39323,ACIDENTE,<NA>,<NA>,CASIMIRO DE ABREU,RJ,BRASIL,<NA>,2010-01-24,16:15:00,<NA>,NaN,NaN,NÃO,NaN,0,1,NÃO
49,39295,39295,39295,39295,39295,ACIDENTE,<NA>,<NA>,RIO DE JANEIRO,RJ,BRASIL,<NA>,2010-02-06,21:09:00,<NA>,NaN,NaN,NÃO,NaN,0,1,NÃO
149,40447,40447,40447,40447,40447,ACIDENTE,-23.0208333333,-44.0955555556,MANGARATIBA,RJ,BRASIL,<NA>,2010-04-21,18:55:00,<NA>,FINALIZADA,A DEFINIR,NÃO,NaN,0,1,NÃO
155,40554,40554,40554,40554,40554,ACIDENTE,-22.4761111111,-44.4711111111,RESENDE,RJ,BRASIL,<NA>,2010-04-27,18:47:00,SIM,FINALIZADA,A-114/CENIPA/2012,SIM,2012-10-17,8,1,NÃO
248,41729,41729,41729,41729,41729,ACIDENTE,<NA>,<NA>,RIO DE JANEIRO,RJ,BRASIL,<NA>,2010-06-25,13:05:00,<NA>,NaN,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5412,79309,79309,79309,79309,79309,ACIDENTE,-22.6575858,-43.6534343,RIO DE JANEIRO,RJ,BRASIL,<NA>,2020-04-17,14:30:00,SIM,ATIVA,A-051/CENIPA/2020,NÃO,NaN,0,1,NÃO
5504,79430,79430,79430,79430,79430,ACIDENTE,-22.8916395,-43.184235,RIO DE JANEIRO,RJ,BRASIL,<NA>,2020-07-06,17:40:00,NÃO,ATIVA,A-081/CENIPA/2020,NÃO,NaN,0,1,NÃO
5637,79621,79621,79621,79621,79621,ACIDENTE,-22.703056,-43.994722,RIO CLARO,RJ,BRASIL,<NA>,2020-10-15,11:00:00,SIM,ATIVA,A-129/CENIPA/2020,NÃO,NaN,0,1,NÃO
5717,79724,79724,79724,79724,79724,ACIDENTE,-23.0217630,-44.5276677,ANGRA DOS REIS,RJ,BRASIL,<NA>,2020-12-11,21:15:00,NÃO,ATIVA,A-149/CENIPA/2020,NÃO,NaN,0,1,NÃO


In [20]:
#filter3 = df.ocorrencia_classificacao.isin(["ACIDENTE","INCIDENTE"]) #Filtra todas as classificacoes de ocorrencia que tiverem essas strings []
#filter4 = df.ocorrencia_cidade.str[-2:] == "RO" #Filtra todas as cidades que terminam com as duas últimas letras "RO" #-2: é o passo
#filter5 = df.ocorrencia_cidade.str.contains("M") #Filtra todas as cidades que possuirem "MA" no nome
filtroData = df.ocorrencia_dia.dt.year == 2018 #Filtra todas as linhas do ano de 2018
df.loc[filtroData]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
4304,77607,77607,77607,77607,77607,INCIDENTE,-164.380.555.55,-390.838.888.88,PORTO SEGURO,BA,BRASIL,SBPS,2018-01-03,11:36:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
4305,77608,77608,77608,77608,77608,INCIDENTE GRAVE,-321.761.111.11,-528.988.888.88,ARROIO GRANDE,RS,BRASIL,<NA>,2018-01-03,13:51:00,SIM,NaN,NaN,NÃO,NaN,0,1,NÃO
4306,77610,77610,77610,77610,77610,INCIDENTE GRAVE,-228.227.777.77,-432.477.777.77,RIO DE JANEIRO,RJ,BRASIL,SBGL,2018-01-04,23:38:00,SIM,ATIVA,NaN,NÃO,NaN,0,1,NÃO
4307,77616,77616,77616,77616,77616,INCIDENTE GRAVE,<NA>,<NA>,PORTO SEGURO,BA,BRASIL,<NA>,2018-01-04,20:45:00,SIM,NaN,NaN,NÃO,NaN,0,1,NÃO
4308,77617,77617,77617,77617,77617,INCIDENTE,-204.694.444.44,-546.702.777.77,CAMPO GRANDE,AL,BRASIL,SBCG,2018-01-04,20:44:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4743,78410,78410,78410,78410,78410,INCIDENTE,-234.355.555.55,-464.730.555.55,GUARULHOS,SP,BRASIL,SBGR,2018-12-27,03:15:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
4744,78390,78390,78390,78390,78390,ACIDENTE,-10.8875,-37.093611111,ARACAJU,SE,BRASIL,<NA>,2018-12-29,15:30:00,SIM,FINALIZADA,A-192/CENIPA/2018,SIM,2019-05-16,1,1,NÃO
4745,78433,78433,78433,78433,78433,INCIDENTE,-255.316.666.66,-491.761.111.11,CURITIBA,PR,BRASIL,SBCT,2018-12-29,14:30:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO
4746,78406,78406,78406,78406,78406,INCIDENTE,-234.355.555.55,-464.730.555.55,GUARULHOS,SP,BRASIL,<NA>,2018-12-30,17:30:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO


In [ ]:
filter_dia_inicial = (df.ocorrencia_dia > 2) & (df.ocorrencia_dia < 30)

In [25]:
#Criando uma nova coluna com duas colunas juntas
df ["data_hora"] = pd.to_datetime(df.ocorrencia_dia.astype(str) + " " + df.ocorrencia_hora)
df.loc[:, ["data_hora"]]

,data_hora
0,2010-01-03 12:00:00
1,2010-01-03 11:05:00
2,2010-01-03 03:00:00
3,2010-01-04 17:30:00
4,2010-01-05 19:25:00
...,...
6109,2021-08-15 15:05:00
6110,2021-08-16 02:30:00
6111,2021-08-17 15:45:00
6112,2021-08-18 16:00:00


In [29]:
#Filtrando os recultados da nova coluna
filter_periodo_inicial = df.data_hora >= "2018-01-01 00:00:00"
filter_periodo_final = df.data_hora >= "2018-12-31 23:59:59"
df.loc[filter_periodo_inicial & filter_periodo_final]

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,data_hora
4748,78391,78391,78391,78391,78391,ACIDENTE,-234.580.555.55,-450.630.555.55,UBATUBA,SP,BRASIL,<NA>,2019-01-01,12:55:00,SIM,ATIVA,NaN,NÃO,NaN,0,1,NÃO,2019-01-01 12:55:00
4749,78396,78396,78396,78396,78396,INCIDENTE GRAVE,<NA>,<NA>,TUCURUÍ,PA,BRASIL,<NA>,2019-01-01,20:10:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2019-01-01 20:10:00
4750,78397,78397,78397,78397,78397,ACIDENTE,-271.497.222.22,-48.615,PORTO BELO,SC,BRASIL,SDEN,2019-01-01,14:38:00,SIM,FINALIZADA,A-001/CENIPA/2019,SIM,2019-10-28,0,1,SIM,2019-01-01 14:38:00
4751,78411,78411,78411,78411,78411,INCIDENTE GRAVE,-8.12638889,-34.92277778,RECIFE,PE,BRASIL,SBRF,2019-01-02,12:20:00,NaN,FINALIZADA,IG-003/CENIPA/2019,SIM,2021-03-29,3,2,NÃO,2019-01-02 12:20:00
4752,78522,78522,78522,78522,78522,INCIDENTE GRAVE,-23.783611111,-52.3875,ENGENHEIRO BELTRÃO,PR,BRASIL,<NA>,2019-01-03,19:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2019-01-03 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6109,80245,80245,80245,80245,80245,INCIDENTE,-8.12638,-34.92277,RECIFE,PE,BRASIL,SBRF,2021-08-15,15:05:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2021-08-15 15:05:00
6110,80247,80247,80247,80247,80247,INCIDENTE,-8.713611111111,-63.90277777777,PORTO VELHO,RO,BRASIL,SBPV,2021-08-16,02:30:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2021-08-16 02:30:00
6111,80251,80251,80251,80251,80251,INCIDENTE,-1.384722222222,-48.47888888888,BELÉM,PA,BRASIL,SBBE,2021-08-17,15:45:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2021-08-17 15:45:00
6112,80257,80257,80257,80257,80257,ACIDENTE,-15.430833,-54.699722,DOM AQUINO,MT,BRASIL,<NA>,2021-08-18,16:00:00,SIM,FINALIZADA,<NA>,NÃO,NaN,0,1,NÃO,2021-08-18 16:00:00


Carregamento de Dados

In [ ]:
#Filtros finais para carregamento
###Não eliminar colunas se não for extremamente necessário. Se a quantidade de campos nulos ou inconsistentes for muito grande,
# é melhor apenas normalizalos para "NA", pra não perder dados.

filtro_ano = df.ocorrencia_dia.dt.year == 2012
filtro_mes = df.ocorrencia_dia.dt.month == 3

#Carregando filtros em um dataframe

df_2012marco = df.loc[filtro_ano & filtro_mes]
df_2012marco

In [39]:
#filtrando o novo dataframe #Essa forma de filtragem não diz se existem valores nulos

#df_2012marco.groupby(["ocorrencia_classificacao"]).ocorrencia_classificacao.count()

#ou

#df_2012marco.groupby(["ocorrencia_classificacao"]).size()

#ou (ordenado)

df_2012marco.groupby(["ocorrencia_classificacao"]).size().sort_values(ascending=True)

ocorrencia_classificacao
INCIDENTE GRAVE     4
ACIDENTE           14
INCIDENTE          43
dtype: int64

In [ ]:
filtro2011 = df.ocorrencia_dia.dt.year == 2012
filtroSudeste = df.ocorrencia_uf.isin(["SP","RJ","MG","ES"])

df_sudeste = df.loc[filtro2011 & filtroSudeste]
df_sudeste

In [43]:
df_sudeste.groupby(["ocorrencia_classificacao"]).size()

ocorrencia_classificacao
ACIDENTE            86
INCIDENTE          188
INCIDENTE GRAVE     30
dtype: int64

In [44]:
df_sudeste.groupby(["ocorrencia_uf","ocorrencia_classificacao"]).size() #Uma lista pode ser passada

ocorrencia_uf  ocorrencia_classificacao
ES             ACIDENTE                     1
               INCIDENTE                   11
MG             ACIDENTE                    21
               INCIDENTE                   37
               INCIDENTE GRAVE              9
RJ             ACIDENTE                    14
               INCIDENTE                   53
               INCIDENTE GRAVE              5
SP             ACIDENTE                    50
               INCIDENTE                   87
               INCIDENTE GRAVE             16
dtype: int64

In [46]:
df_sudeste.groupby(["ocorrencia_cidade","ocorrencia_classificacao"]).size().sort_values(ascending=False)

ocorrencia_cidade  ocorrencia_classificacao
RIO DE JANEIRO     INCIDENTE                   27
SÃO PAULO          INCIDENTE                   24
BELO HORIZONTE     INCIDENTE                   21
MACAÉ              INCIDENTE                   15
GUARULHOS          INCIDENTE                   12
                                               ..
MOGI DAS CRUZES    INCIDENTE                    1
MONTES CLAROS      INCIDENTE                    1
OLIVEIRA           ACIDENTE                     1
OURO FINO          INCIDENTE GRAVE              1
AGUDOS             INCIDENTE                    1
Length: 122, dtype: int64

In [ ]:
df_sudeste.loc[df_sudeste.ocorrencia_uf == "MG"]

Carregamento para CSV

In [54]:
df_sudeste.to_csv("sudeste.csv", index=False) #dataframe.tipo_arquivo("nome_nova_tabela_csv")

In [55]:
teste = pd.read_csv("/content/sudeste.csv") 
#Para não ser criado um rótulo "Unnamed" deve se colocar um nome da coluna, ou inserir "index=False" antes das exportacao
teste

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,ocorrencia_pais,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista,data_hora
0,45331,45331,45331,45331,45331,ACIDENTE,-23.435556,-46.473056,GUARULHOS,SP,BRASIL,SBGR,2012-01-06 00:00:00,13:44:00,SIM,FINALIZADA,A-582/CENIPA/2014,SIM,2016-09-01,3,1,NÃO,2012-01-06 13:44:00
1,45401,45401,45401,45401,45401,ACIDENTE,NaN,NaN,SÃO SEBASTIÃO,SP,BRASIL,NaN,2012-01-06 00:00:00,17:00:00,NaN,NaN,NaN,NÃO,NaN,0,1,NÃO,2012-01-06 17:00:00
2,52243,52243,52243,52243,52243,INCIDENTE,NaN,NaN,UBATUBA,SP,BRASIL,NaN,2012-01-06 00:00:00,14:30:00,NaN,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2012-01-06 14:30:00
3,50713,50713,50713,50713,50713,INCIDENTE GRAVE,NaN,NaN,CAMPINAS,SP,BRASIL,SDAI,2012-01-07 00:00:00,18:15:00,SIM,FINALIZADA,IG-184/CENIPA/2013,SIM,2014-01-08,0,1,NÃO,2012-01-07 18:15:00
4,45391,45391,45391,45391,45391,ACIDENTE,-19.913333,-48.293056,CONCEIÇÃO DAS ALAGOAS,MG,BRASIL,NaN,2012-01-08 00:00:00,16:00:00,NaN,FINALIZADA,A DEFINIR,NÃO,NaN,0,1,NÃO,2012-01-08 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,46793,46793,46793,46793,46793,INCIDENTE,NaN,NaN,SÃO PAULO,SP,BRASIL,NaN,2012-12-28 00:00:00,16:30:00,NaN,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2012-12-28 16:30:00
300,46780,46780,46780,46780,46780,ACIDENTE,NaN,NaN,SANTA FÉ DO SUL,SP,BRASIL,NaN,2012-12-29 00:00:00,21:00:00,NaN,NaN,NaN,NÃO,NaN,0,1,NÃO,2012-12-29 21:00:00
301,46814,46814,46814,46814,46814,INCIDENTE,NaN,NaN,BELO HORIZONTE,MG,BRASIL,SBBH,2012-12-29 00:00:00,15:10:00,NaN,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO,2012-12-29 15:10:00
302,46839,46839,46839,46839,46839,ACIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,BRASIL,NaN,2012-12-29 00:00:00,19:00:00,NaN,ATIVA,A-503/CENIPA/2021,NÃO,NaN,0,1,NÃO,2012-12-29 19:00:00
